# 1️⃣ C++’ta hata tipleri (neyle uğraşıyoruz?)

### Genel resim:

* Derleme zamanı hatası (compile-time)

* Sözdizimi bozuk, tip uyumsuz, eksik ; vs.

* Derleyici bağırır, program hiç oluşmaz.

### Çalışma zamanı hatası (runtime)

* Dosya yok, model dosyası bozuk, new hafıza veremiyor, sıfıra bölme…

* Derleyici bunu önceden göremez.

* Burada exception / hata kodu / abort üçlüsünden biri devreye girer.

### Undefined behavior (UB)

* Dangling pointer, out-of-bounds erişim, double free vs.

* Program patlayabilir, saçma sonuç verebilir, sessizce yanlış davranabilir.


Bizim hata yönetimi dediğimiz kısım, esas olarak 2. ve kısmen 3. maddeyi yönetme savaşı.

---
# 2️⃣ throw – “Burada işler boka sardı, devam edemem”

* C++’ta exception fırlatmak için throw kullanırsın.

Basit örnek:
```cpp
double bol(double a, double b) {
    if (b == 0.0) {
        throw std::runtime_error("Sıfıra bölme hatası");
    }
    return a / b;
}
```



* Burada throw şunu diyor:
>“Bu fonksiyonda devam edemem, akışı yukarıya fırlatıyorum. Biri beni yakalasın.”

Bu tipikler:

* std::runtime_error

* std::logic_error

* std::invalid_argument

* std::out_of_range

* std::bad_alloc (hafıza yetersizliği, new atar)

vs.

**Hepsi std::exception’dan türeyen sınıflar.**

---

# 3️⃣ try–catch – “Yukarıdan gelen patlamayı ben yakalarım”

* Exception fırlatma tarafı throw.

Yakalama tarafı:
```cpp
try {
    // riskli kod
    double sonuc = bol(10, 0);
    std::cout << "Sonuç: " << sonuc << "\n";
}
catch (const std::runtime_error& e) {
    std::cerr << "Hata: " << e.what() << "\n";
}
catch (const std::exception& e) {
    std::cerr << "Genel hata: " << e.what() << "\n";
}
```

Mantık:

* try bloğunda exception fırlarsa (örneğin bol(10, 0)),
* try içindeki geri kalan kod çalışmaz, direkt uygun catch bloğuna zıplar.

* catch (const std::runtime_error& e)
**→ throw std::runtime_error(...) gibi spesifik hataları yakalar.**

* catch (const std::exception& e)
**→ std::exception ailesindeki tüm hatalar için genel bir ağ.**

Sıra önemli: önce spesifik yakalayıcılar, sonra genel.



----
# 4️⃣ Stack unwinding – “Arada kimler ölüyor?”

**Exception fırlayınca C++’ın yaptığı şey: stack unwinding.**

Yani:

* funcA() içinden funcB() çağırdın

* funcB() içinde bir yerde throw oldu

* funcB()’den çıkarken otomatik değişkenlerin destructors’ı çalışır

* Sonra funcA() seviyesine çıkılır

* Orada bir try–catch yoksa bir üst seviyeye, en son main’e kadar çıkar

* Hiçbir yerde yakalanmazsa std::terminate() → program göçer

Bu yüzden RAII ve destructor mantığı önemli:
>Çünkü exception gelse bile, block’tan çıkarken kaynaklar toplanır.

### Örnek
```cpp

struct Kaynak {
    Kaynak() { std::cout << "Kaynak açıldı\n"; }
    ~Kaynak() { std::cout << "Kaynak kapandı\n"; }
};

void test() {
    Kaynak k;

    throw std::runtime_error("Patladık");
    std::cout << "Burası hiç çalışmayacak\n";
}

int main() {
    try {
        test();
    } catch (const std::exception& e) {
        std::cerr << "Yakalandı: " << e.what() << "\n";
    }
}
```

#### Çıktı:
```bash
Kaynak açıldı
Kaynak kapandı
Yakalandı: Patladık
```

>Dikkat: Exception fırlamasına rağmen ~Kaynak() çalışıyor.
**İşte RAII olayı; smart pointer’lar, std::vector vs hep bu yüzden kullanılıyor.**


---
# 5️⃣ “Exception mi, return error code mu?”

C++ dünyasında iki ana ekol var:

* Exception bazlı:

>Hata olursa throw.

>Normal akış ile hata akışını ayırırsın.

>LibTorch, OpenCV, STL’in büyük kısmı bu kafada.

* Error code bazlı:

> int dönersin; 0 = başarı, başka değer = hata.

> C tarzı API’lerde bu yaygın (soketler, bazı OS API’leri vs.)

> Biz LibTorch + OpenCV tarafında olacağımız için:

> Exception kullanımını öğrenmek zorundayız,

**ama yanında error code tarzı yaklaşımla da karşılaşacağız.**

---
# 6️⃣ Hata yönetimini katmanlı düşün

Özellikle LibTorch/OpenCV ile gerçek proje yazarken mantık şöyle olmalı:

* En iç katman (fonksiyonlar)

* Dosya bulunamadı, tensor shape yanlış, model load başarısız vs.

* Buralarda exception fırlatmak mantıklı.

* Orta katman (uygulama mantığı)

Try–catch ile hatayı yakalayıp:

* Logla

* Kullanıcıya düzgün mesaj ver

* Devam mı et, yoksa programı mı kapat karar ver

En dış katman (main)

* En son savunma hattı:
**“Aşağıdan kaçan exception varsa, bari burada düzgün loglayıp temizce çıkayım.”**

### Örnek main kalıbı:
```cpp
int main() {
    try {
        // program giriş noktası
        run_app();
    }
    catch (const std::exception& e) {
        std::cerr << "[FATAL] Beklenmeyen hata: " << e.what() << "\n";
        return EXIT_FAILURE;
    }
    catch (...) {
        std::cerr << "[FATAL] Bilinmeyen türde exception yakalandı.\n";
        return EXIT_FAILURE;
    }

    return EXIT_SUCCESS;
}
```


---

# 8️⃣ try–catch nerede kullanılmalı, nerede kullanılmamalı?

Genel akıl:

* Mikro seviyede her yere try–catch sokma, kod çöp olur.

Mantıklı blokları sar:

* model load kısmı

* main loop (kamera–inference)

* config dosyası okuma vs.

---
---

----
----

# 1️⃣ Re-throw: İçeride logla, yukarıya pasla

Bazen şöyle bir durum istiyorsun:

* Fonksiyonun içinde hafif bir işlem yapıp log atmak istiyorsun

* Ama hatayı burada yiyip bitirmek istemiyorsun

* Üst seviye de bilsin, belki programı kapatsın vs.

Bu durumda:

```cpp
catch (...) {
    // burada logla, cleanup yap, sonra tekrar fırlat
    throw;
}
```


* throw; (dikkat: parametresiz)
* → “aynı exception’ı bir üst seviyeye tekrar fırlat” demek.


### Mini örnek:

```cpp
void araKatman() {
    try {
        riskliFonksiyon();
    }
    catch (const std::exception& e) {
        std::cerr << "[araKatman log] Hata: " << e.what() << "\n";
        throw; // Aynı exception'ı tekrar yukarı fırlat
    }
}

int main() {
    try {
        araKatman();
    }
    catch (const std::exception& e) {
        std::cerr << "[main] Son savunma: " << e.what() << "\n";
    }
}
```

Akış:

* riskliFonksiyon patlatır → araKatman’daki catch’e düşer

* araKatman log yazar, sonra throw; ile aynı hatayı main’e gönderir

* main de yakalayıp “global” olarak ele alır

Bu pattern ileride şuna dönüşecek:

* runInference() içinde LibTorch’tan gelen hatayı loglayıp tekrar fırlatmak

* main() seviyesinde de “fatal” olarak işlemek

# 2️⃣ catch(...) – Türü bile bilmediğin her şeyi yakala

* C++’ta türü bilmediğin veya bilmek istemediğin her şeyi yakalamak için:

```cpp
catch (...) {
    std::cerr << "Bilinmeyen bir exception yakalandı\n";
}
```

* Bu genelde en en dış seviyede, son savunma hattı olarak kullanılı

### Örnek:
```cpp
int main() {
    try {
        run_app();
    }
    catch (const std::exception& e) {
        std::cerr << "[main] std::exception: " << e.what() << "\n";
    }
    catch (...) {
        std::cerr << "[main] Türü bilinmeyen bir exception yakalandı\n";
    }
}
```


**LibTorch/OpenCV gibi kütüphaneler çoğunlukla std::exception türevleri fırlatır ama nadir edge-case’lerde veya kendi yazdığın saçmalıklarda türü belli olmayan şeyler de uçabilir. catch(...) “son çöp ağı”dır.**